In [4]:
# libraries

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from time import sleep
import re

# MongoConnection
from pymongo import MongoClientu

##  Currently working for 

In [ ]:
# City :

# Parali, India

In [ ]:
# url :

# https://www.wunderground.com/history/wmo/42953/2018/4/18/DailyHistory.html?req_city=Kanker&req_state=CT&req_statename=India&reqdb.zip=00000&reqdb.magic=33&reqdb.wmo=42953

## *1.1 ) SCRAPE USING Requests BeautifulSoup :*

In [5]:
# CONNECTION TO MONGODB
# MAKE CONNECTION AS REQUIRED ACCORING TO THE CITY/SUBSTATION TO PROCESS :

# CONNECTION TO MONGO :

client = MongoClient()
db_DDForecast = client['db_DDForecast']
collection_WeatherUndergroud_Bahur = db_DDForecast['collection_WeatherUndergroud_Bahur']  # if success collection
collection_WeatherUndergroud_fail_Bahur = db_DDForecast['collection_WeatherUndergroud_fail_Bahur']  # failed collection

# success - data is successfully found from the day url
# fail - data table not found from the day url, may be because of request failure or data not uploaded for that day

In [6]:
def getMyColumnsOfTable(div_table_target):
    '''
    Fn to find all the column names of the table concerned - BS Version
    '''
    # finding the table heads 
    column_row = div_table_target.findAll('tr')[0]  # all parameters obtained in a list
    
    # list of parameters
    data_parameters = [ re.sub(r"\.","",each_param.text) for each_param in column_row.findAll('th')] 
    return data_parameters

In [7]:

# VERSION IN USE

range_year = np.arange(2008, 2020)  # 2008 to 2019
range_month = np.arange(1, 13)  # 1 to 12


# once we reach today's date, then break whatsoever 
flag_break_today = None

for each_year_current in range_year:
    
    for each_month_current in range_month:
        
        # finding the total no of days in that month
        total_days_in_this_month = pd.to_datetime( str(each_month_current) + " " + str(each_year_current)).days_in_month
        
        
        for current_day_of_month in range(1, total_days_in_this_month + 1):
            today = pd.to_datetime('today')
                        
            if current_day_of_month == (int(today.day) ) and each_month_current == today.month and each_year_current == today.year:
                
                # will be breaking from every loop
                flag_break_today = 1
                print('Breaking at today ', today)
                print("year - ", today.year, "month - ", today.month, "date - ", today.day )
                break           
            
            
            
       #   https://www.wunderground.com/history/wmo/43329/2018/6/6/DailyHistory.html?req_city=Bahur&req_state=PY&req_statename=India&reqdb.zip=00000&reqdb.magic=1&reqdb.wmo=43329



            
            url = 'https://www.wunderground.com/history/wmo/43329/'+str(each_year_current)+'/'+str(each_month_current)+'/'+str(current_day_of_month)+'/DailyHistory.html?req_city=Bahur&req_state=PY&req_statename=India&reqdb.zip=00000&reqdb.magic=1&reqdb.wmo=43329'
            
            timeStamp = pd.Timestamp(day=current_day_of_month, month=each_month_current, year=each_year_current)
            TimeStamp_self_create = str(timeStamp.day_name()) + ", " + str(timeStamp.month_name()) + " " + str(timeStamp.day) + ", " + str(timeStamp.year)
            
        #   if collection_WeatherUnderground_Parali.find_one({'Timestamp':TimeStamp_self_create}) or collection_WeatherUnderground_fail_Parali.find_one({'failed_timestamp':TimeStamp_self_create}):
            if collection_WeatherUndergroud_Bahur.find_one({'url':url}) or collection_WeatherUndergroud_fail_Bahur.find_one({'failed_url':url}):
                # if url is present already in db
                print("already inserted in success or failed collection")
                continue

                
            try:
                sleep(4)
                ua = UserAgent()
                user_agent = {'User-agent': ua.random}
                
                url_request = requests.get(url, headers=user_agent, timeout=5)    
                soup = BeautifulSoup(url_request.text, 'lxml')
                
                # finding our target table 
                div_table_target = soup.find('div', attrs={"id":"observations_details"}).table    
            
            except:
                collection_WeatherUndergroud_fail_Bahur.insert_one({"failed_url":url})
#                 collection_WeatherUnderground_fail_Parali.insert_one({"failed_url":url, "failed_timestamp":TimeStamp_self_create})
                print('NO DATA PRESENT - URL FAILED : GOES INTO FAILED COLLECTION ')
                continue
            
            
            
            # TABLE ROW WISE DICT EXTRACTION :
            # finding the total no of rows in this table 
            total_no_of_rows = len(div_table_target.findAll('tr'))             
            
            # EXTRACT PARAMETER IN EACH ROW AND FRAME A DICTIONARY OUT OF THAT:
            for i in range(1, total_no_of_rows):
                
                # finding all the td i.e. 'table data' elements inside that current row
                current_row_containing_tds = div_table_target.findAll('tr')[i]
                
                # start preparing a dictionary row wise containing elements of the table
                dict_row_current = {}
                dict_row_current['url'] = url

                timeStamp = pd.Timestamp(day=current_day_of_month, month=each_month_current, year=each_year_current)
                TimeStamp_self_create = str(timeStamp.day_name()) + ", " + str(timeStamp.month_name()) + " " + str(timeStamp.day) + ", " + str(timeStamp.year)
                
                # framing a timestamp using year, month, day, dayname (not scraping the timestamp as it creates problems)
                dict_row_current['Timestamp'] = TimeStamp_self_create
                
                # dict_row_current['Timestamp'] = soup.find('h2', attrs={"class":"history-date"}).text
                
                # list of all keys that the dictionary shall contain i.e. table column names
                table_parameters_list = getMyColumnsOfTable(div_table_target)
                
                # Dictionary will get ready in the following loop for a row elements:
                try:
                    for j in range(0, len(table_parameters_list)):
                        dict_row_current[table_parameters_list[j]] = current_row_containing_tds.findAll('td')[j].text
                except:
                    continue
                
                # dictionary ready for insertion :
                # this row as dict shall be inserted into the db
                print("inserting....")
                collection_WeatherUndergroud_Bahur.insert_one(dict_row_current)

                
        if flag_break_today == 1:
            print("BREAKING 1")
            break
    
    if flag_break_today == 1:
        print("BREAKING 2 - final")
        break

NO DATA PRESENT - URL FAILED : GOES INTO FAILED COLLECTION 
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
in

inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
insert

inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
insert

inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
insert

inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
insert

...

inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
insert

inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
insert

inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
insert

inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
insert

inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
inserting....
NO DATA PRESENT - URL FAILED : GOES INTO FAILED COLLECTION 
NO DATA PRESENT - URL FAILED : GOES INTO FAILED COLLECTION 
NO DATA PRESENT - URL FAILED : GOES INTO FAILED COLLECTION 
NO DATA PRESENT - URL FAILED : GOES INTO FAILED COLLECTION 
Breaking at today  2019-06-20 17:55:33.099453
year -  2019 month -  6 date -  20
BREAKING 1
BREAKING 2 - final


> <h3> + UNDER THE BS EXCEL SHEET, MARK 'SCRAPED' = 1 FOR THIS CITY </h3>

## *1.2 ) Iterate for each failed url present in failed collection for the city:(BS TYPE ONLY)*

### *For failed urls of BS type, reiterate through them using 'Beautifulsoup and requests' again  :*

In [ ]:
# # CONNECTION TO MONGODB
# DEFINE AS PER EACH CITY PROCESSING

# client = MongoClient()
db_DDForecast = client['db_DDForecast']   # Mongo DB 
collection_WeatherUndergroud_Parali = db_DDForecast['collection_WeatherUndergroud_Parali']  # DB collection 
collection_WeatherUndergroud_fail_Parali = db_DDForecast['collection_WeatherUndergroud_fail_Parali']

In [ ]:
# failed_urls_list = []

# for each in collection_WeatherUnderground_fail_urmal.find():
#     failed_urls_list.append(each['failed_url'])

In [ ]:
# PREPARE A LIST OF FAILED URLS OF BS TYPE, USING THE CORRESPONDING FAILED COLLECTION OF CITY
# TRY TO FIND DATA FROM THESE FAILED URLS, 
# IF DATA FOUND, ENTER INTO SUCCESS COLLECTION AND DELETE THIS URL FROM FAILED COLLECTION CORESPODING


failed_urls_list = []

for each in collection_WeatherUnderground_fail_Parali.find():
    failed_urls_list.append(each['failed_url'])


# reversing the failed list    
failed_urls_list.reverse()    

# failed_urls_list = list(set(failed_urls_list))

# initiate the driver    
# driver = webdriver.Chrome()
# driver.maximize_window()

for each_failed_url in failed_urls_list:
            
#     open this failed_url through webdriver 
#     driver.get(each_failed_url)
    

    try:
        sleep(4)
        user_agent = {'User-agent': 'Mozilla/5.0'}
        responseBS = requests.get(each_failed_url, headers=user_agent, timeout=20)
#       soup = BeautifulSoup(driver.page_source, 'lxml')
        soup = BeautifulSoup(responseBS.text, 'lxml')
        sleep(2.5)
    
        # finding our target table 
        div_table_target = soup.find('div', attrs={"id":"observations_details"}).table    

    except:
        print('STILL NOT DATA IN THIS URL : ', each_failed_url)
        # do nothing and let the failed url be present in the fail_collection,,
        # incase data table is not present on the url then nothing can be done, as this is finally marked failed url atLast
        continue



    # TABLE DATA - ROW WISE DICT EXTRACTION BEGINS ----> :
    # finding the total no of rows in this table 
    total_no_of_rows = len(div_table_target.findAll('tr'))             

    # EXTRACT PARAMETER IN EACH ROW AND FRAME A DICTIONARY OUT OF THAT:
    for i in range(1, total_no_of_rows):

        # Finding all the td i.e. 'table data' elements inside that current row
        current_row_containing_tds = div_table_target.findAll('tr')[i]

        # start preparing a dictionary row wise containing elements of the table
        dict_row_current = {}
        dict_row_current['url'] = each_failed_url
        dict_row_current['Timestamp'] = soup.find('h2', attrs={"class":"history-date"}).text

        # list of all keys that the dictionary shall contain i.e. table column names
        table_parameters_list = getMyColumnsOfTable(div_table_target)

        # Dictionary will get ready in the following loop for a row elements:
        try:
            for j in range(0, len(table_parameters_list)):
                dict_row_current[table_parameters_list[j]] = current_row_containing_tds.findAll('td')[j].text
        except:
            continue

        # dictionary ready for insertion :
        # this row as dict shall be inserted into the db
        print("inserting....")

        collection_WeatherUnderground_Parali.insert_one(dict_row_current)  # on success enter the dictionaries one by one into success collection 
        
        # delete the record of failed link  from the fail_collection as wanted data has been obtained from this URL:
        collection_WeatherUnderground_fail_Parali.delete_one({'failed_url':each_failed_url})

# driver.close()
# driver.quit()

## *1.3) CLEAN - BS4 BASED :* 

## export colletion as csv

In [ ]:
alora_df.loc[0, 'Temp'].encode('ascii',errors='ignore').decode()

In [ ]:
def clean_Pressure(pressure_value_unclean):
    
    pressure = pressure_value_unclean.encode('ascii',errors='ignore').decode()
#     pressure = ''.join(list(filter(lambda x: x in printable, pressure)))
    pressure = re.sub(r"\\n|\\t", '', pressure)
    pressure_clean = re.sub(r"hPa", "", pressure)
    return pressure_clean

def clean_Temperature(temperature_value_unclean):
    
    temperature = temperature_value_unclean.encode('ascii',errors='ignore').decode()
#     temperature = ''.join(list(filter(lambda x: x in printable, temperature)))
    temperature = re.sub(r"\\n|\\t", '', temperature)
    temperature = re.sub(r"-", "", temperature)
    temperature = re.sub(r" ", "", temperature)
    temperature = re.sub(r"   ", "", temperature)
    temperature_clean = re.sub(r"C", "", temperature)
    return temperature_clean

def clean_Humidity(humidity_value_unclean):
    
    humidity = humidity_value_unclean.encode('ascii',errors='ignore').decode()
#     humidity = ''.join(list(filter(lambda x: x in printable, humidity)))
    humidity = re.sub(r"\\n|\\t", '', humidity)
    humidity = re.sub(r"%", "", humidity)
    humidity = re.sub(r" ","", humidity)
    humidity_clean = re.sub(r"   ", "", humidity)
    return humidity_clean

In [ ]:
# loading the unclean csv :

urmal_df = pd.read_csv(r"E:\Codes At Intern\Genesis Ray Energy\Demand Forecasting Model\WeatherUnderground Data Evidence Check for Cities\Data Cities Unclean After Scraping\WeatherUnderground_Aranmula_unclean.csv")
urmal_df.head(3)

In [ ]:
# cleaning the Temperature, Pressure, Humidity parameters :

urmal_df['Temp2'] = urmal_df['Temp'].apply(clean_Temperature)
urmal_df['Pressure2'] = urmal_df['Pressure'].apply(clean_Pressure)
urmal_df['Humidity2'] = urmal_df['Humidity'].apply(clean_Humidity)

In [ ]:
urmal_df.head()

In [ ]:
# Empty string values to be converted to 9999 :

urmal_df.loc[urmal_df['Temp2'] == '', 'Temp2'] = 99999
urmal_df.loc[urmal_df['Pressure2'] == '', 'Pressure2'] = 99999
urmal_df.loc[urmal_df['Humidity2'] == '', 'Humidity2'] = 99999

In [ ]:
# Creating Pandas timestamp :

urmal_df['Complete TimeStamp'] = urmal_df['Time (IST)'] + " " + urmal_df['Timestamp']
urmal_df['Complete TimeStamp'] = pd.to_datetime(urmal_df['Complete TimeStamp'])
urmal_df.head(3)

In [ ]:
# Type conversion of the  Temperature , Pressure and Humidity parameters :

urmal_df['Temp2'] = urmal_df['Temp2'].astype('float')
urmal_df['Pressure2'] = urmal_df['Pressure2'].astype('float')
urmal_df['Humidity2'] = urmal_df['Humidity2'].astype('float')

In [ ]:
# output the dataframe as csv :

urmal_df.to_csv(r"E:\Codes At Intern\Genesis Ray Energy\Demand Forecasting Model\WeatherUnderground Data Evidence Check for Cities\Data Cities Clean After Scraping\Aranmula_clean_final2.csv", 
                        index=False,
                        columns=['Temp2', 'Humidity2', 'Pressure2', 'Complete TimeStamp'])

## *2.1) SCRAPE USING Selenium, (Ex in City = 'vijayawada')*

In [ ]:
def getMyColumnsOfTable(div_table_target):
    '''
    FN to get data table column names when using selenium format of scraping
    ''' 
    
    # finding the table heads 
    column_row = div_table_target.findAll('tr')[0]  # all parameters obtained in a list
    
    # list of parameters
    data_parameters = [ re.sub(r"\.","",each_param.text) for each_param in column_row.findAll('button')] 
    return data_parameters

In [ ]:
client = MongoClient()
db_DDForecast = client['db_DDForecast']   # Mongo DB 
collection_WeatherUndergroud_begumpet = db_DDForecast['collection_WeatherUndergroud_begumpet']  # success collection name
collection_WeatherUndergroud_fail_begumpet = db_DDForecast['collection_WeatherUndergroud_fail_begumpet'] # fail collection name

In [ ]:
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--incognito")

driver = webdriver.Chrome()
driver.maximize_window()

In [ ]:
# VERSION IN USE

# CONNECTION TO MONGODB for begumpet


range_year = np.arange(2018, 2020)  # 2008 to 2019
range_month = np.arange(1, 13)  # 1 to 12


# once we reach today's date, then break whatsoever 
flag_break_today = None

for each_year_current in range_year:
    
    for each_month_current in range_month:
        
        # finding the total no of days in that month
        total_days_in_this_month = pd.to_datetime( str(each_month_current) + " " + str(each_year_current)).days_in_month
        
        
        for current_day_of_month in range(1, total_days_in_this_month + 1):
            
            today = pd.Timestamp('today')  # get todays' day, month and year
            current_loop_date = pd.Timestamp(year=each_year_current, month=each_month_current, day=current_day_of_month) # get current looping's day, month and year              
            
            if (today - current_loop_date).days < 0 :   # if (today-tomorrow) gives -ve days as time interval then break
                
                # will be breaking from every loop
                flag_break_today = 1
                print('Breaking at today ', today)
                print("year - ", today.year, "month - ", today.month, "date - ", today.day )
                break           
            
            
            # https://www.wunderground.com/history/daily/VOHY/date/2018-4-10?req_city=Zahirabad&req_state=AP&req_statename=India&reqdb.zip=00000&reqdb.magic=6&reqdb.wmo=43083
            url = 'https://www.wunderground.com/history/daily/VOHY/date/' + str(each_year_current) + '-' + str(each_month_current) + '-' + str(current_day_of_month) + "?req_city=Zahirabad&req_state=AP&req_statename=India&reqdb.zip=00000&reqdb.magic=6&reqdb.wmo=43083"                        
            
            
            if collection_WeatherUndergroud_begumpet.find_one({"url":url}) or collection_WeatherUndergroud_fail_begumpet.find_one({"failed_url":url}):
                # if url is present alread in db
                print("already present in success db ")
                continue
                

            try:
                
                driver.get(url)   # open url through webddriver Chrome
                driver.maximize_window()   # maximize window
                sleep(10)  # sleep for 10 seconds
                driver.find_element_by_tag_name('body').send_keys(Keys.END)
                driver.execute_script('window.scrollBy(0, -100);')
                driver.execute_script('window.scrollBy(0, 50);')
                driver.execute_script('window.scrollBy(0, -50);')
                sleep(2)
                soup = BeautifulSoup(driver.page_source, 'lxml')
                
                # finding our target table 
                div_table_target = soup.find('div', attrs={"class":"observation-table"}).table     
            
            except:
                collection_WeatherUndergroud_fail_begumpet.insert_one({"failed_url":url})
                continue
            
            
            
            # TABLE ROW WISE DICT EXTRACTION :
            # finding the total no of rows in this table 
            total_no_of_rows = len(div_table_target.findAll('tr'))             
            
            # EXTRACT PARAMETER IN EACH ROW AND FRAME A DICTIONARY OUT OF THAT:
            for i in range(1, total_no_of_rows):
                
                # finding all the td i.e. 'table data' elements inside that current row
                current_row_containing_tds = div_table_target.findAll('tr')[i]
                
                # start preparing a dictionary row wise containing elements of the table
                dict_row_current = {}
                dict_row_current['url'] = url
                
                # obtaining the month :
                month_name = driver.find_element_by_xpath(r"//div[@class='date-selector']//select[1]")
                value_month_name = month_name.get_attribute('value')
                dict_row_current['Month_value'] = driver.find_element_by_xpath(r"//div[@class='date-selector']//select[1]//option[@value=" + value_month_name + ']').text
                
                # obtaining the date :
                date_name = driver.find_element_by_xpath(r"//div[@class='date-selector']//select[2]")
                dict_row_current['Date_value'] = date_name.get_attribute('value')
                
                # obtaining the year :
                year_name = driver.find_element_by_xpath(r"//div[@class='date-selector']//select[3]")
                dict_row_current['year name'] = year_name.get_attribute('value')

                
                # list of all keys that the dictionary shall contain i.e. table column names
                table_parameters_list = getMyColumnsOfTable(div_table_target)
                
                # Dictionary will get ready in the following loop for a row elements:
                
                try:
                    for j in range(0, len(table_parameters_list)):
                        dict_row_current[table_parameters_list[j]] = current_row_containing_tds.findAll('td')[j].text
                except:
                    continue
                    
                
                # dictionary ready for insertion :
                # this row as dict shall be inserted into the db
                print("inserting....")
                collection_WeatherUndergroud_begumpet.insert_one(dict_row_current)

        if flag_break_today == 1:
            print("BREAKING 1")
            break
    
    if flag_break_today == 1:
        print("BREAKING 2 - final")
        break
    
                
driver.quit()

## *2.2) Iterate for each failed url present in failed collection for the city:(SELENIUM URLS TYPE ONLY)*

### *For failed urls of Selenium type, reiterate through them using selenium :*

In [ ]:
# PREPARE A LIST OF FAILED URLS OF BS TYPE, USING THE CORRESPONDING FAILED COLLECTION OF CITY
# TRY TO FIND DATA FROM THESE FAILED URLS, 
# IF DATA FOUND, ENTER INTO SUCCESS COLLECTION AND DELETE THIS URL FROM FAILED COLLECTION CORESPODING


failed_urls_list = []

for each in collection_WeatherUndergroud_fail_vijayawada.find():
    failed_urls_list.append(each['failed_url'])

failed_urls_list.reverse()    
    
# initiate the driver    
driver = webdriver.Chrome()
driver.maximize_window()

for each_failed_url in failed_urls_list:
            
    # open this failed_url through webdriver 
    driver.get(each_failed_url)
    sleep(4)

    try:

        driver.maximize_window()   # maximize window
        sleep(9)  # sleep for 10 seconds
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        driver.execute_script('window.scrollBy(0, -100);')
        driver.execute_script('window.scrollBy(0, 50);')
        driver.execute_script('window.scrollBy(0, -50);')
        sleep(2)
        soup = BeautifulSoup(driver.page_source, 'lxml')

        # finding our target table 
        div_table_target = soup.find('div', attrs={"class":"observation-table"}).table     

    except:
#         collection_WeatherUndergroud_fail_vijayawada.insert_one({"failed_url":url})
        continue




    # TABLE DATA - ROW WISE DICT EXTRACTION BEGINS ----> :
    # finding the total no of rows in this table 
    total_no_of_rows = len(div_table_target.findAll('tr'))             

    # EXTRACT PARAMETER IN EACH ROW AND FRAME A DICTIONARY OUT OF THAT:
    for i in range(1, total_no_of_rows):

        # Finding all the td i.e. 'table data' elements inside that current row
        current_row_containing_tds = div_table_target.findAll('tr')[i]

        # start preparing a dictionary row wise containing elements of the table
        dict_row_current = {}
        dict_row_current['url'] = each_failed_url
        dict_row_current['Timestamp'] = soup.find('h2', attrs={"class":"history-date"}).text

        # list of all keys that the dictionary shall contain i.e. table column names
        table_parameters_list = getMyColumnsOfTable(div_table_target)

        # Dictionary will get ready in the following loop for a row elements:
        
        try:
            for j in range(0, len(table_parameters_list)):
                dict_row_current[table_parameters_list[j]] = current_row_containing_tds.findAll('td')[j].text
        except:
            continue


        # dictionary ready for insertion :
        # this row as dict shall be inserted into the db
        print("inserting....")

        collection_WeatherUndergroud_vijayawada.insert_one(dict_row_current)  # on success enter the dictionaries one by one into success collection 
        
        # delete the record of failed link  from the fail_collection as wanted data has been obtained from this URL:
        collection_WeatherUndergroud_fail_vijayawada.delete_one({'failed_url':each_failed_url})

driver.close()
driver.quit()

## *2.3) CLEAN - SELENIUM BASED :* 

In [ ]:
def clean_Pressure(pressure_value_unclean):
    
#     pressure = pressure_value_unclean.encode('ascii',errors='ignore').decode()
#     pressure = ''.join(list(filter(lambda x: x in printable, pressure)))
    pressure = re.sub(r"\\n", '', pressure_value_unclean)
    pressure = re.sub(r"-", "", pressure)
    pressure = re.sub(r" ", "", pressure)
    pressure = re.sub(r"   ", "", pressure)
    pressure_clean = re.sub(r"in", "", pressure)
    return pressure_clean

def clean_Temperature(temperature_value_unclean):
    
#     temperature = temperature_value_unclean.encode('ascii',errors='ignore').decode()
#     temperature = ''.join(list(filter(lambda x: x in printable, temperature)))
    temperature = re.sub(r"\\n|\\t", '', temperature_value_unclean)
    temperature = re.sub(r"-", "", temperature)
    temperature = re.sub(r" ", "", temperature)
    temperature = re.sub(r"   ", "", temperature)
    temperature_clean = re.sub(r"C|F", "", temperature)
    return temperature_clean

def clean_Humidity(humidity_value_unclean):
    
#     humidity = humidity_value_unclean.encode('ascii',errors='ignore').decode()
#     humidity = ''.join(list(filter(lambda x: x in printable, humidity)))
    humidity = re.sub(r"\\n|\\t", '', humidity_value_unclean)
    humidity = re.sub(r"%", "", humidity)
    humidity = re.sub(r" ","", humidity)
    humidity = re.sub(r"  ","", humidity)
    humidity_clean = re.sub(r"   ", "", humidity)
    return humidity_clean


def clean_Time(time_value_unclean):
    time = re.sub(r"\\n","",time_value_unclean)
    time = re.sub(r" ","",time)
    time = re.sub(r"  ","",time)
    time_clean = re.sub(r"   ","",time)
    return time_clean 


def clean_Month_Value(month_value_unclean):
    month = re.sub(r"\\n", "", month_value_unclean)
    month = re.sub(r" ", "", month)
    month = re.sub(r"  ", "", month)
    month_clean = re.sub(r"   ", "", month)
    return month_clean 

    # date clean by default
    # year clean by default

In [ ]:
# loading the unclean csv :

vijayawada_df = pd.read_csv(r"E:\Codes At Intern\Genesis Ray Energy\ForecastingModel - WeatherUnderground_Data\Unclean_Cities_kurnool_kakinada_tirupati_vishkhap\db_DDForecast.collection_WeatherUndergroud_vijayawada_unclean.csv")
vijayawada_df.head(3)

In [ ]:
# cleaning the Temperature, Pressure, Humidity parameters :

vijayawada_df['Temp2'] = vijayawada_df['Temperature'].apply(clean_Temperature)
vijayawada_df['Pressure2'] = vijayawada_df['Pressure'].apply(clean_Pressure)
vijayawada_df['Humidity2'] = vijayawada_df['Humidity'].apply(clean_Humidity)
vijayawada_df['Time'] = vijayawada_df['Time'].apply(clean_Time)
vijayawada_df['Month_value'] = vijayawada_df['Month_value'].apply(clean_Time)

In [ ]:
# Empty values to be converted to 99999 :
# but no empty values exist for vijaywada city  :

vijayawada_df.loc[vijayawada_df['Temp2'] == '', 'Temp2'] = 9999
vijayawada_df.loc[vijayawada_df['Pressure2'] == '', 'Pressure2'] = 9999
vijayawada_df.loc[vijayawada_df['Humidity2'] == '', 'Humidity2'] = 9999

In [ ]:
# Creating Pandas timestamp :

vijayawada_df['Time'] = vijayawada_df['Time'].astype('str')
vijayawada_df['Date_value'] = vijayawada_df['Date_value'].astype('str')
vijayawada_df['Month_value'] = vijayawada_df['Month_value'].astype('str')
vijayawada_df['year name'] = vijayawada_df['year name'].astype('str')

vijayawada_df['Complete TimeStamp'] = vijayawada_df['Time'] + " " + vijayawada_df['Date_value'] + " " + vijayawada_df['Month_value'] + " " + vijayawada_df['year name']
vijayawada_df['Complete TimeStamp'] = pd.to_datetime(vijayawada_df['Complete TimeStamp'])
vijayawada_df.head(3)

In [ ]:
# Type conversion of the  Temperature , Pressure and Humidity parameters :

vijayawada_df['Temp2'] = vijayawada_df['Temp2'].astype('float')
vijayawada_df['Pressure2'] = vijayawada_df['Pressure2'].astype('float')
vijayawada_df['Humidity2'] = vijayawada_df['Humidity2'].astype('float')

In [ ]:
# output the dataframe as csv :

vijayawada_df.to_csv(r"E:\Codes At Intern\Genesis Ray Energy\ForecastingModel - WeatherUnderground_Data\Clean_Cities_kurnool_kakinada_tirupati_vishkhap\vijayawada_clean_final.csv", 
                   index=False, 
                   columns=['Temp2', 'Humidity2', 'Pressure2', 'Complete TimeStamp'])

In [ ]:
# CHECK FOR NaN

vijayawada_df[['Temp2', 'Humidity2', 'Pressure2']].isnull().sum()